In [ ]:
import os
import re
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from db.models import (StreetType, CityType, Address, BelpostAddress,
                    get_database_engine, get_database_url)
from parser import setup_driver, search_postal_code
from rapidfuzz import fuzz

## 0. Загрузка данных

In [2]:
path = os.path.join("private_data", "ko_adresses.xlsx")
data = pd.read_excel(path)
data.head()

,to.street,to.building,soato.imns,soato.oblast,soato.district,soato.sovet,soato.tip,soato.name
0,УЛ. КОЛЬЦЕВАЯ,3,621,Минская,NaN,NaN,г.,Солигорск
1,УЛ. ВЛАДИМИРА ЛЕНИНА,238-3А,313,Витебская,NaN,NaN,г.,Орша
2,УЛ. ИНОГОРОДНЯЯ 3-Я,35,400,NaN,NaN,NaN,г.,Гомель
3,УЛ. БОРИСОВСКАЯ,2Г,603,Минская,Борисовский,Гливинский,д.,Гора
4,"УЛ. СУДОСТРОИТЕЛЬНАЯ, 10, пом. 316",NaN,419,Гомельская,Речицкий,Озерщинский,д.,Озерщина


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264997 entries, 0 to 264996
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   to.street       260015 non-null  object
 1   to.building     228084 non-null  object
 2   soato.imns      264997 non-null  int64 
 3   soato.oblast    134385 non-null  object
 4   soato.district  43071 non-null   object
 5   soato.sovet     39320 non-null   object
 6   soato.tip       264997 non-null  object
 7   soato.name      264997 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.2+ MB


## 1. Предобработка данных 

### 1.1. Замена аббревиатур населенных на полные слова

In [4]:
data["soato.tip"].value_counts()

soato.tip
г.     221616
аг.     13737
д.      13087
гп       6520
с/с      6246
р-н      1687
п.       1620
рп        246
кп        212
х.         24
пгт         2
Name: count, dtype: int64

In [5]:
replace_dict = {
    "г.": "город",
    "аг.": "агрогородок",
    "гп": "городской поселок",
    "д. ": "деревня",
    "с/с": "сельский совет",
    "р-н": "район",
    "п.": "поселок",
    "рп": "рабочий поселок",
    "кп": "курортный поселок",
    "х.": "хутор",
    "пгт": "поселок городского типа",
}

### 1.2. Универсальный конструктор адреса для поиска

In [6]:
def build_address(
    region: str = None,
    district: str = None,
    sovet: str = None,
    city_type: str = None,
    city_name: str = None,
    street: str = None,
    building: str = None,
) -> str:
    """
    Универсальный конструктор адреса.
    Пропускает отсутствующие поля, не добавляя лишних слов.
    """

    parts = []

    if region:
        parts.append(f"{region} область")

    if district:
        parts.append(f"{district} район")

    if sovet:
        parts.append(f"{sovet} сельсовет")

    if city_name:  # тип города добавляем только если есть название
        if city_type:
            parts.append(f"{city_type} {city_name}")
        else:
            parts.append(city_name)

    if street:
        parts.append(street)

    if building:
        parts.append(building)

    return ", ".join(parts)

### 1.3 Замена аббревиатур в названиях улиц (поле `to.street`)

In [7]:
# Извлекаем возможные сокращения
pattern = r"^([\w\-/.]+)\s"
data["abbr"] = data["to.street"].str.extract(pattern)
abbrs = (data["abbr"].dropna().unique())
abbrs
abbrs_with_dot_dash = [a for a in abbrs if re.search(r'[.\-]', a)]
abbrs_no_digits = [abbr for abbr in abbrs_with_dot_dash if not re.search(r"\d", abbr)]
data["abbr"].value_counts()[:10], abbrs_no_digits[:5]

(abbr
 УЛ.      206234
 ПР.       22615
 ПЕР.       6141
 ШОССЕ      2643
 Б-Р        1969
 ПЛ.        1873
 ТР.        1812
 ул.        1083
 МКР-Н       866
 ПР-Д        796
 Name: count, dtype: int64,
 ['УЛ.', 'ПР.', 'ПЕР.', 'ул.', 'ПЛ.'])

In [8]:
# Загружаем JSON-справочник
abbr_dict = dict()
with open('abbrs_deepseek.json', 'r', encoding='utf-8') as f:
    abbr_dict = json.load(f)
list(abbr_dict.items())[:10]

[('УЛ.', 'улица'),
 ('ПР.', 'проспект'),
 ('ПЕР.', 'переулок'),
 ('ул.', 'улица'),
 ('ПЛ.', 'площадь'),
 ('ТР.', 'тракт'),
 ('пр-т', 'проспект'),
 ('ПР-Д', 'проезд'),
 ('аг.', 'агрогородок'),
 ('Б-Р', 'бульвар')]

In [9]:
def expand_abbreviations_in_column(df, column, abbr_dict):
    """
    Заменяет аббревиатуры в столбце DataFrame на полные слова по справочнику.
    Игнорирует регистр, работает и для случаев без пробела между аббревиатурой и словом.
    abbr_dict: dict, где ключ — аббревиатура, значение — расшифровка.
    """
    # Сортируем аббревиатуры по убыванию длины, чтобы длинные сначала заменялись
    abbrs = sorted(abbr_dict.keys(), key=len, reverse=True)
    # Составляем паттерн для поиска всех аббревиатур (с учётом возможного отсутствия пробела)
    patterns = []
    for abbr in abbrs:
        if abbr_dict[abbr]:  # только если есть расшифровка
            # Экранируем спецсимволы, разрешаем отсутствие пробела после аббревиатуры
            pat = r'(?i)\b' + re.escape(abbr) + r'\s*'
            patterns.append((re.compile(pat), abbr_dict[abbr]))

    def replace_abbr(text):
        if not isinstance(text, str):
            return text
        result = text
        for pat, full in patterns:
            # Заменяем только в начале слова или после пробела/начала строки
            result = pat.sub(full + ' ', result)
        # Удаляем двойные пробелы, если появились
        result = re.sub(r'\s+', ' ', result).strip()
        return result

    res = df.copy()
    res[column] = res[column].apply(replace_abbr)
    return res

In [10]:
data = expand_abbreviations_in_column(data, "to.street", abbr_dict)
data.fillna("", inplace=True)
data.sample(10)

,to.street,to.building,soato.imns,soato.oblast,soato.district,soato.sovet,soato.tip,soato.name,abbr
154611,улица 17 СЕНТЯБРЯ,53Д,315,Витебская,,,г.,Поставы,УЛ.
245112,улица ЮРИЯ СЕМЕНЯКО,38,100,,,,г.,Минск,УЛ.
50489,улица ГАСТЕЛЛО,47/2-123,511,Гродненская,,,г.,Лида,УЛ.
137395,улица МАТУСЕВИЧА,64,100,,,,г.,Минск,УЛ.
79906,улица ФЕДЮНИНСКОГО,2,400,,,,г.,Гомель,УЛ.
60428,улица ФРАНЦИСКА СКОРИНЫ,2 кв 31,213,Брестская,Лунинецкий,,г.,Микашевичи,УЛ.
17650,улица БОБРУЙСКАЯ,6,100,,,,г.,Минск,УЛ.
23693,улица СОВЕТСКАЯ,74,319,Витебская,,,гп,Ушачи,УЛ.
43634,улица ПУШКИНСКАЯ,11,200,,,,г.,Брест,УЛ.
143738,улица ТИМИРЯЗЕВА,23,100,,,,г.,Минск,УЛ.


In [11]:
# Проверка на наличие аббревиатур
pattern = r"^([\w\-/.]+)\s"
data["abbr"] = data["to.street"].str.extract(pattern)
data["abbr"].value_counts()[:10]

abbr
улица         209044
проспект       22746
переулок        6275
ШОССЕ           2643
бульвар         2056
площадь         1943
тракт           1817
микрорайон       891
проезд           813
район            738
Name: count, dtype: int64

### 2. Поиск адреса в базе (запрос к `belpost.by`)

In [12]:
# Загрузка драйвера и сессии
engine = get_database_engine(echo=False)
session = Session(engine)
driver = setup_driver()

In [13]:
# Составление адреса запроса и запрос к belpost.by
region = district = sovet = city_type = city_name = street = building = address_str = None
with session.begin():
    idx = 146137
    # addr = session.query(Address).where(Address.id == 121212).first()

    # street = addr.street
    # building = addr.building
    # region = addr.soato_oblast
    # district = addr.soato_district
    # sovet = addr.soato_sovet
    # city_name = addr.soato_name
    # city_type = replace_dict.get(addr.soato_tip, "")
    
    data_row = data.iloc[idx]
    region = data_row["soato.oblast"]
    district = data_row["soato.district"]
    sovet = data_row["soato.sovet"]
    city_type = replace_dict.get(data_row["soato.tip"], "")
    city_name = data_row["soato.name"]
    street = data_row["to.street"]
    building = data_row["to.building"]

    address_str = build_address(
        region=region,
        district=district,
        sovet=sovet,
        city_type=city_type,
        city_name=city_name,
        street=street,
        building=building,
    )

    print("Составленный адрес: ", address_str)

    res = search_postal_code(driver, address_str)
    res = pd.DataFrame(res, columns=["Почтовый код","Область","Район",
                                     "Город","Улица","Номер дома"])
    session.close()
res

Составленный адрес:  Могилевская область, город Осиповичи, улица СТРОИТЕЛЕЙ, 28
Открываем URL: https://www.belpost.by/Uznatpochtovyykod28indek?search=%D0%9C%D0%BE%D0%B3%D0%B8%D0%BB%D0%B5%D0%B2%D1%81%D0%BA%D0%B0%D1%8F%20%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D1%8C%2C%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%20%D0%9E%D1%81%D0%B8%D0%BF%D0%BE%D0%B2%D0%B8%D1%87%D0%B8%2C%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%20%D0%A1%D0%A2%D0%A0%D0%9E%D0%98%D0%A2%D0%95%D0%9B%D0%95%D0%99%2C%2028
Ожидание результатов поиска...


,Почтовый код,Область,Район,Город,Улица,Номер дома
0,213764,Могилевская,Осиповичский,город Осиповичи,улица Строителей,ВСЕ
1,224022,Брестская,Брестский,город Брест,улица Июля 28,"1А, 1Б, 1В, 3А, 3Б, 3В, 3Г"
2,224011,Брестская,Брестский,город Брест,улица Июля 28,"29, 31, 33, 33А, 33Б, 35, 37, 37А, 43, 45, 45/..."
3,224704,Брестская,Брестский,город Брест,улица Июля 28,"20, 28, 28А, 30, 40, 40А, 50"
4,211162,Витебская,Чашникский,город Новолукомль,переулок 28 Центральный,ВСЕ
5,222311,Минская,Молодечненский,деревня Менютки,территория Азс 28,ВСЕ
6,213206,Могилевская,Чаусский,город Чаусы,улица Могилевская,ВСЕ
7,247196,Гомельская,Жлобинский,город Жлобин,улица Могилевская,"1-28, 1А, 2А"
8,211387,Витебская,Оршанский,город Орша,улица Могилевская,"(3-25), 6, 18, 2, 4, 4/1, 4/2, 4/5, (8-16), 22..."
9,211389,Витебская,Оршанский,город Орша,улица Могилевская,"85/1, 85/2, 85/3, 85/4, 87, 89, 91, 93, 85, 85..."


## 3. Фильтрация и поиск похожих адресов

### 3.1. Жёсткая фильтрация (по области, району, городу)

In [14]:
def filter_addresses(df: pd.DataFrame, region: str, district: str, city: str) -> pd.DataFrame:
    """Жёсткая фильтрация DataFrame по региону, району, городу и диапазону домов."""
    mask = (
        (df["Область"].astype(str).str.contains(region, case=False, na=False) if region else True) &
        (df["Район"].astype(str).str.contains(district, case=False, na=False) if district else True) &
        (df["Город"].astype(str).str.contains(city, case=False, na=False) if city else True)
    )
    return df[mask]

In [15]:
first_filtered = filter_addresses(res,
                region=region,
                district=district,
                city=city_name)
print("Поисковый адрес: ", address_str)
first_filtered

Поисковый адрес:  Могилевская область, город Осиповичи, улица СТРОИТЕЛЕЙ, 28


,Почтовый код,Область,Район,Город,Улица,Номер дома
0,213764,Могилевская,Осиповичский,город Осиповичи,улица Строителей,ВСЕ


### 3.2 Поиск по похожести имени улицы (`rapidfuzz`)

In [16]:
def add_similarity_scores(df: pd.DataFrame, target_string: str, column_name: str, scorer=fuzz.ratio):
    """
    Быстрое вычисление схожести с использованием rapidfuzz
    """
    df = df.copy()
    
    # Вычисляем схожесть для всех строк сразу
    scores = [scorer(str(x).lower(), str(target_string).lower()) for x in df[column_name]]
    
    df['similarity_score'] = scores
    df.sort_values(by="similarity_score", ascending=False, inplace=True)
    return df

In [17]:
print("Улица: ", street)
data_scores = add_similarity_scores(first_filtered,
                      target_string=street,
                      column_name="Улица")
data_scores

Улица:  улица СТРОИТЕЛЕЙ


,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score
0,213764,Могилевская,Осиповичский,город Осиповичи,улица Строителей,ВСЕ,100.0


### 3.3. Поиск по номеру дома (содержится ли в списке домов)

In [18]:
def normalize_house(house: str):
    """Разбивает дом на число, букву/корпус (если есть)."""
    house = house.strip().upper()
    # Пример: 71/1 → основа 71, корпус 1
    m = re.match(r"(\d+)([А-ЯA-Z]*)?(?:/(\d+))?", house)
    if not m:
        return None, None, None
    num = int(m.group(1))
    letter = m.group(2) or None
    korp = m.group(3) or None
    return num, letter, korp


def house_in_range(house: str, rule: str) -> bool:
    """Проверка: принадлежит ли дом правилу из списка домов."""
    if not house or not rule:
        return False
    
    house_num, house_letter, house_korp = normalize_house(house)
    if house_num is None:
        return False
    
    rule = rule.strip().upper()
    
    if rule == "ВСЕ":
        return True

    # Разделяем правила через запятую
    parts = [p.strip().upper() for p in rule.split(",")]
    for part in parts:
        # диапазон чёт/нечет
        m = re.match(r"\((\d+)-(\d+)\)", part)
        if m:
            start, end = int(m.group(1)), int(m.group(2))
            if house_num % 2 == start % 2 and start <= house_num <= end:
                return True
            continue

        # обычный диапазон
        m = re.match(r"^(\d+)-(\d+)$", part)
        if m:
            start, end = int(m.group(1)), int(m.group(2))
            if start <= house_num <= end:
                return True
            continue

        # конкретный номер (включая буквы и корпуса)
        if part == house.upper():
            return True

        # # только число (без букв)
        # if part.isdigit() and house_num == int(part):
        #     return True

    return False

In [19]:
target_house = building
print("Номер дома: ", target_house)
data_scores["house_match"] = data_scores["Номер дома"].apply(lambda r: house_in_range(target_house, r))
data_scores

Номер дома:  28


,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score,house_match
0,213764,Могилевская,Осиповичский,город Осиповичи,улица Строителей,ВСЕ,100.0,True


### 3.4. Выбор лучшего результата

In [20]:
res = data_scores[data_scores["house_match"]] # выбираем только те строки, где дом совпал
max_score = res['similarity_score'].max()
res[res['similarity_score'] == max_score] # выбираем строку с максимальным значением к-та совпадения

,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score,house_match
0,213764,Могилевская,Осиповичский,город Осиповичи,улица Строителей,ВСЕ,100.0,True
